In [19]:
import numpy as np
import pandas as pd

## Utility functions

In [3]:
def get_station_updates(city, station):
    df = pd.read_csv('challenge_data/{}/stations/{}.csv'.format(city, station))
    df['moment'] = pd.to_datetime(df['moment'])
    return df


def get_weather_updates(city):
    df = pd.read_csv('challenge_data/{}/weather.csv'.format(city))
    df['moment'] = pd.to_datetime(df['moment'])
    return df

## Feature extraction

In [44]:
def extract_time_features(df):
    df['weekday'] = df['moment'].apply(lambda m: m.weekday())
    df['hour'] = df['moment'].apply(lambda m: m.hour)
    df['minute'] = df['moment'].apply(lambda m: m.minute)

def add_weather_updates(city_df, city_name):
    weather_df = get_weather_updates(city_name)
    city_df.index = np.searchsorted(weather_df['moment'], city_df['moment'])
    joined_df = pd.merge(left=city_df, right=weather_df, left_index=True, right_index=True, how='right')
    joined_df.rename(columns={'moment_x': 'moment', 'moment_y': 'moment_weather'}, inplace=True)
    return joined_df


df = get_station_updates('toulouse', '00003-pomme')
df = add_weather_updates(df, 'toulouse')
df

,moment,bikes,spaces,moment_weather,clouds,description,humidity,pressure,temperature,wind
0,2016-04-01 00:08:18,8,10,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:09:18,8,10,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:20:19,6,12,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:21:18,5,13,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:22:18,3,14,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:23:21,3,14,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:23:56,2,14,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:25:13,3,14,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:25:23,4,13,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50
0,2016-04-01 00:32:20,4,13,2016-04-02 00:01:58,0,clear sky,81,1017.00,3.15,1.50


## Let's go HAM